In [ ]:
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import shapely.geometry
import shapely.ops
import pandas as pd
import folium
import earthpy
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import json
from ipywidgets import HTML
import numpy as np
import multiprocessing
import time
from functools import partial


### Buffering parks and clipping rivers

In [ ]:
parks = gpd.read_file("../../data/rivers/Polygon_layer.shp").to_crs(epsg=4269)


for index, park in parks.iterrows():
    print(park.loc['name'])


park_name = 'Eastern CAR Wilderness - no settlements no agriculture - 102\'000 km2'


# Buffer
park_buffer = parks.loc[parks['name'] == park_name].to_crs('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')

park_buffer['geometry'] = park_buffer['geometry'].geometry.buffer(500000)
park_buffer = park_buffer.to_crs(epsg=4269)

# Park
park = parks.loc[parks['name'] == park_name].to_crs('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')
park['geometry'] = park['geometry'].geometry.buffer(3000)

park = park.to_crs(epsg=4269)

# Rivers
try:
    rwa_rivers
except NameError:
    rwa_rivers = gpd.read_file("../../data/rivers/clipped_rivers.shp").to_crs(epsg=4269)

# Buffer rivers
buffer_border = gpd.GeoDataFrame(geometry=park_buffer.boundary)
buffer_rivers = gpd.sjoin(rwa_rivers, park_buffer, how='inner', predicate='within')

# Park rivers
park_rivers = gpd.sjoin(rwa_rivers, park, how='inner', predicate='within')

park_rivers = park_rivers.reset_index(drop=True)

# Set property if in park
buffer_rivers['in_park'] = 0
for index1, river1 in buffer_rivers.iterrows():
    if river1['NOID'] in park_rivers['NOID'].values:
        buffer_rivers.at[index1, 'in_park'] = 1
        
buffer_rivers = buffer_rivers.rename(columns={'index_left': 'i_l'})
buffer_rivers = buffer_rivers.rename(columns={'index_right': 'i_r'})
buffer_rivers = buffer_rivers.reset_index(drop=True)

buffer_rivers['park_dis']=0.0
buffer_rivers['calc_dis']=0.0
buffer_rivers['up_dis']=0.0
buffer_rivers['ran']=0
buffer_rivers['in_ran']=0

buffer_rivers['start_dis']=0.0
buffer_rivers['outside_dis']=0.0
buffer_rivers['enters_park'] = 0
buffer_rivers['inflow']=0
buffer_rivers['through_flow']=0

# Rivers on boundary of park
buffer_rivers['intersect_park'] = 0
park_rivers['intersect_park'] = 0
park_rivers['outside_dis']=0.0

park_border = gpd.GeoDataFrame(geometry=park.boundary)
all_inters = gpd.sjoin(buffer_rivers, park_border, op='intersects')



for index1, river1 in buffer_rivers.iterrows():
    if river1['NOID'] in all_inters['NOID'].values:
        buffer_rivers.at[index1, 'intersect_park'] = 1
        buffer_rivers.at[index1, 'in_park'] = 1

for index1, river1 in buffer_rivers.iterrows():
    if river1['NOID'] in park_rivers['NOID'].values:
        buffer_rivers.at[index1, 'in_park'] = 1
        

for index1, river1 in all_inters.iterrows():
    if river1['NOID'] in park_rivers['NOID'].values:
        park_rivers.at[index1, 'intersect_park'] = 1  
        

all_inters = all_inters.reset_index(drop=True)        
inters = gpd.GeoDataFrame()
for index1, r1 in all_inters.iterrows():
    already_intersected = 0
    for index2, r2 in all_inters.iterrows():
        if r1['NOID'] == r2['NDOID']:
            already_intersected = 1
    if already_intersected != 1:
        inters = inters.append(all_inters.iloc[index1])
            
# Make df of all rivers inside and intersecting park
in_park = park_rivers.append(inters)



fig, ax = plt.subplots()
ax.set_aspect('equal')
buffer_rivers.plot(ax=ax, color='grey')
park_border.plot(ax=ax, color='red')
in_park.plot(ax=ax, color='orange')

# inters.plot(ax=ax, color='blue')
plt.show();



### Find upstream river outside park

In [ ]:
in_inters = gpd.GeoDataFrame()
inters = inters.reset_index(drop=True)

# Loop through all rivers intersecting with park border
for index1, r1 in inters.iterrows():
    # All rivers starting
    if r1['NUOID'] is None:
        pass
    # All rivers not starting
    else:
        for index2, r2 in buffer_rivers.loc[buffer_rivers['NDOID']==r1['NOID']].iterrows():
            # If river (r2) flows into river on border (r1)
            if r1['NOID'] == r2['NDOID']:
                # to make sure upstream river (r2) is outside the park
                if r2['in_park'] != 1:
                    # then upstream river (r2) is flowing into the park
                    r1['inflow'] = 1
                    in_inters = in_inters.append(inters.iloc[index1])
                    
            if r1['NOID'] == r2['NOID']:
                buffer_rivers.at[index2, 'inflow'] = 1


if in_inters.shape[0] > 0:                

    # drop all duplicate in-flowing rivers
    in_inters = in_inters.drop_duplicates()
    print('Number of inflowing rivers: ' + str(len(in_inters)))

    # set property on buffer_rivers to know that river is entering the park from outside
    for index1, river1 in buffer_rivers.iterrows():
        if river1['NOID'] in in_inters['NOID'].values:
            buffer_rivers.at[index1, 'enters_park'] = 1  

    # plot to show in-flowing rivers
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    park_border.plot(ax=ax, color='grey')
    park_rivers.plot(ax=ax, color='black')
    inters.plot(ax=ax, color='blue')
    in_inters.plot(ax=ax, color='red')
    plt.show();



### Calculating outside flow

In [ ]:
if in_inters.shape[0] > 0:                

    for index1, r1 in in_inters.iterrows(): 
        print("\n")
        print(r1['NOID'])
        print('start_flow: ' + str(r1['DIS_AV_CMS'] ))


        run = 0
        # If the river has reached the park boundary and is now flowing out
        out_flow = 0
        new_inters = gpd.GeoDataFrame()

        # While the river is still inside the park
        while out_flow == 0:
            run += 1
            print('run' + str(run))
            # If in-flowing rivers on park boundary
            if run == 1:
                for index2, r2 in buffer_rivers.loc[buffer_rivers['NOID']==r1['NDOID']].iterrows():
                    # If inflowing river (r1) is flowing into downstream river (r2)
                    if r1['NDOID'] == r2['NOID']:

                        # Set the downstream river (r2)'s outside flow to the inflowing river (r1)'s flow
                        buffer_rivers.at[index2, 'outside_dis'] = r2['outside_dis']  + r1['DIS_AV_CMS']

                        # Mark r2 as flowing through the park
                        buffer_rivers.at[index2, 'through_flow'] = 1
                        # Add r2 to become the next round's upstream river
                        new_inters = new_inters.append(buffer_rivers.iloc[index2])
                        # Check if r2 is already crossing the park boundary (leaving the park)
                        if r2['in_park'] == 0:
                            out_flow = 1

            # If not the in-flowing river, but a next-in-line downstream river

            else:
                for index3, r3 in new_inters.iterrows():
                    new_inters = gpd.GeoDataFrame()
                    for index4, r4 in buffer_rivers.loc[buffer_rivers['NOID']==r3['NDOID']].iterrows():
                        # If upstream river (r3) is flowing into downstream river (r4)                    
                        if r3['NDOID'] == r4['NOID']:
                            buffer_rivers.at[index4, 'outside_dis'] = r4['outside_dis']  + r1['DIS_AV_CMS']

                            buffer_rivers.at[index4, 'through_flow'] = 1
                            new_inters = new_inters.append(buffer_rivers.iloc[index4])
                            if r4['in_park'] == 0:
                                out_flow = 1                       



### Calculate park contribution

In [ ]:
for index1, r1 in buffer_rivers.iterrows():
    if r1['outside_dis'] != 0:
        buffer_rivers.at[index1, 'park_dis'] = r1['DIS_AV_CMS'] - r1['outside_dis']
        buffer_rivers.at[index1, 'calc_dis'] = (r1['DIS_AV_CMS'] - r1['outside_dis'])/r1['DIS_AV_CMS']



### Combine with rivers starting inside park

In [ ]:
# Loop through all rivers intersecting with park border
for index1, r1 in inters.iterrows():
#     buffer_rivers = buffer_rivers.reset_index()
    buffer_rivers['ran']=0
    buffer_rivers['up_dis']=0.0
    inters['start_dis']=inters['DIS_AV_CMS']
    buffer_rivers['start_dis']=0.0 
    
    # All rivers starting
    if r1['NUOID'] is None:
        buffer_rivers['calc_p']=0.0

# If inter set buffer_rivers to 1
for index1, r1 in inters.iterrows():    
    for index2, r2 in buffer_rivers.loc[buffer_rivers['NOID']==r1['NOID']].iterrows():
        if r1['NOID'] == r2['NOID'] and r2['enters_park'] == 0:
            buffer_rivers.at[index2, 'calc_p'] = 1
    for index2, r2 in buffer_rivers.loc[buffer_rivers['NOID']==r1['NDOID']].iterrows():        
        if (r1['NDOID'] == r2['NOID']) and (r1['NUOID'] is None):
            buffer_rivers.at[index2, 'start_dis'] = r2['start_dis'] + r1['DIS_AV_CMS']

# Set start dis from river flowing out of park
# for index1, r1 in inters.iterrows():
#     for index2, r2 in buffer_rivers.loc[buffer_rivers['NOID']==r1['NOID']].iterrows():
#         if r1['NDOID'] == r2['NOID']:
#             inters.at[index1, 'start_dis'] = r2['start_dis']          
            
new_inters = inters
print(inters.shape[0])
runs = 500
run = 0
while run < 100:
    run += 1
    print(str(round(run*100/runs)))
    new_new_inters = new_inters
    new_inters = gpd.GeoDataFrame()
    for index3, r3 in new_new_inters.iterrows():

        
        if r3['NUOID'] is None:
            buffer_rivers.loc[buffer_rivers['NOID']==r3['NOID'],'start_dis'] = r3['DIS_AV_CMS']            

        else:

            upstream_ids = map(int, r3['NUOID'].split('_'))                
            upstream = buffer_rivers.loc[buffer_rivers['NOID'].isin(upstream_ids)].reset_index()

            buffer_rivers.at[index3, 'start_dis'] = upstream['start_dis'].sum()
            if r4['calc_dis'] == 0:
                buffer_rivers.at[index3, 'park_dis'] = upstream['park_dis'].sum()

        for index4, r4 in buffer_rivers.loc[buffer_rivers['NOID']==r3['NDOID']].iterrows():
                if (r3['NDOID'] == r4['NOID']):

#                     if buffer_rivers.at[index4, 'outside_dis'] == 0:
#                         buffer_rivers.at[index4, 'outside_dis'] = r3['outside_dis']

#                     if r4['calc_dis'] == 0:
#                         buffer_rivers.at[index4, 'park_dis'] = r3['park_dis']
#                         buffer_rivers.at[index4, 'calc_dis'] = r3['calc_dis'] + buffer_rivers.at[index4, 'calc_dis']


#                     buffer_rivers.at[index4, 'calc_p'] = round(buffer_rivers.at[index4, 'start_dis']/r4['DIS_AV_CMS'],4)

                    new_inters = new_inters.append(buffer_rivers.iloc[index4])
                    

# def calc_park_dis(row):
#     if row['outside_dis'] > 0:
#         park_dis = row['DIS_AV_CMS'] - row['outside_dis']
#     else:
#         park_dis = 0
#     return park_dis

    
# buffer_rivers['park_dis'] = buffer_rivers.apply(calc_park_dis, axis=1)
# print(buffer_rivers.head())

buffer_rivers['calc_dis'] = buffer_rivers['park_dis']/buffer_rivers['DIS_AV_CMS']

buffer_rivers['calc_p'] = buffer_rivers['calc_p'] + buffer_rivers['calc_dis']


In [ ]:
buffer_rivers['calc_dis'] = buffer_rivers['park_dis']/buffer_rivers['DIS_AV_CMS']
buffer_rivers['calc_p'] = (buffer_rivers['start_dis']/buffer_rivers['DIS_AV_CMS'])+ buffer_rivers['calc_dis']


In [ ]:
# for index1, r1 in buffer_rivers.iterrows():
#     if buffer_rivers.at[index1, 'in_park'] == 1 and buffer_rivers.at[index1, 'through_flow'] != 1 and buffer_rivers.at[index1, 'enters_park'] == 0:
#         buffer_rivers.at[index1, 'calc_p'] = 1
#     if buffer_rivers.at[index1, 'park_dis'] < 0:
#         buffer_rivers.at[index1, 'park_dis'] = 0
#         buffer_rivers.at[index1, 'calc_p'] = 0  

# relevant_rivers = buffer_rivers.loc[buffer_rivers['calc_p'] > 0]

In [ ]:
relevant_rivers = buffer_rivers.loc[buffer_rivers['calc_p'] > 0]
m = folium.Map([park.centroid.y, park.centroid.x],
                  zoom_start=10,
                  tiles='Stamen Terrain')

def create_cols(dis):
    if dis < 0.1:
        col = '#3e5946'
    else:
        col = '#34278f'
    return col

relevant_rivers['col'] = pd.cut(relevant_rivers['calc_p'], bins=5, labels=['white', '#73a4af', '#4f84a2', '#253c5e', '#182749'])


gj = folium.GeoJson(
    relevant_rivers,
    style_function=lambda feature: {
        'fillColor': feature['properties']['col'],
        'color' : feature['properties']['col'],
        'weight' : 3,
        'fillOpacity' : 0.5,
        },
    popup=folium.GeoJsonPopup(fields=['NOID', 'NDOID', 'NUOID', 'through_flow', 'in_park', 'park_dis', 'calc_p', 'calc_dis', 'start_dis', 'DIS_AV_CMS', 'outside_dis'])
    )

park_plot = folium.GeoJson(
    park,
    style_function=lambda feature: {
        'fillColor': 'red',
        'color' : 'red',
        'weight' : 3,
        'fillOpacity' : 0.5,
        }
    )



park_plot.add_to(m)
gj.add_to(m)
m
